## Load and Plot Results

In [1]:
import pandas as pd

depth_and_num_params_csv_path = 'depth_and_num_params.csv'
evaluate_csv_path = 'evaluate.csv'
problem_scale = 4

# 设置显示选项
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 输入路径
depth_and_num_params_csv_path = 'depth_and_num_params.csv'
evaluate_csv_path = 'evaluate.csv'
problem_scale = 4

# 读入 Depth 和 Params 数据
df = pd.read_csv(depth_and_num_params_csv_path)
df.loc[df["method"] == "HeaSolver", "method"] = "HEA"
df.loc[df["method"] == "PenaltySolver", "method"] = "P-QAOA"
df.loc[df["method"] == "ChocoSolver", "method"] = "Choco-Q"
df.loc[df["method"] == "RasenganSolver", "method"] = "Rasengan"

benchmarks = ["F", "K", "J", "S", "G"]
method_order = ['HEA', 'P-QAOA', 'Choco-Q', 'Rasengan']

# pkid → Benchmark label，如 F1, F2, ..., G4
pkid_to_label = {
    pkid: f"{b}{i}"
    for pkid, (b, i) in enumerate(
        ((b, i) for b in benchmarks for i in range(1, problem_scale + 1))
    )
}
df["Benchmark"] = df["pkid"].map(pkid_to_label)
df["method"] = pd.Categorical(df["method"], categories=method_order, ordered=True)

# 聚合
df_grouped = df.groupby(["method", "Benchmark"], observed=True).mean(numeric_only=True).reset_index()

# 构建 pivot 表格
pivot_depth = df_grouped.pivot(index="method", columns="Benchmark", values="depth")
pivot_param = df_grouped.pivot(index="method", columns="Benchmark", values="num_params")
pivot_depth.loc["Rasengan"] = pivot_depth.loc["Rasengan"] / pivot_param.loc["Rasengan"]

# ARG 数据处理
df_eval = pd.read_csv(evaluate_csv_path)
df_eval.loc[df_eval["method"] == "HeaSolver", "method"] = "HEA"
df_eval.loc[df_eval["method"] == "PenaltySolver", "method"] = "P-QAOA"
df_eval.loc[df_eval["method"] == "ChocoSolver", "method"] = "Choco-Q"
df_eval.loc[df_eval["method"] == "RasenganSolver", "method"] = "Rasengan"
df_eval = df_eval[df_eval['ARG'] <= 100000]
df_eval["Benchmark"] = df_eval["pkid"].map(pkid_to_label)
df_eval["method"] = pd.Categorical(df_eval["method"], categories=method_order, ordered=True)
df_arg_grouped = df_eval.groupby(["method", "Benchmark"], observed=True)["ARG"].mean().reset_index()
pivot_arg = df_arg_grouped.pivot(index="method", columns="Benchmark", values="ARG")

# 列顺序对齐
column_order = list(pkid_to_label.values())
pivot_depth = pivot_depth[column_order]
pivot_param = pivot_param[column_order]
pivot_arg = pivot_arg[column_order]

# 四舍五入

# 多级索引
depth_labeled = pivot_depth.copy()
depth_labeled.index = pd.MultiIndex.from_product([["Depth"], depth_labeled.index])
param_labeled = pivot_param.copy()
param_labeled.index = pd.MultiIndex.from_product([["#Params"], param_labeled.index])
arg_labeled = pivot_arg.copy()
arg_labeled.index = pd.MultiIndex.from_product([["ARG"], arg_labeled.index])

# === 计算 improvement ===
def compute_improvement(target_df, baseline="Rasengan"):
    improvements = {}
    for method in method_order:
        if method == baseline:
            improvements[method] = None
            continue
        ratio = target_df.loc[method] / target_df.loc[baseline]
        improvements[method] = ratio.mean()
    return improvements

def append_improvement_column(df, improvements):
    df = df.copy()
    df["improvement"] = [
        round(val, 3) if val is not None else pd.NA
        for val in [improvements.get(method, pd.NA) for method in df.index.get_level_values(1)]
    ]
    return df

# 分别计算 improvement 并添加列
arg_improvements = compute_improvement(pivot_arg)
depth_improvements = compute_improvement(pivot_depth)
param_improvements = compute_improvement(pivot_param)

depth_labeled = depth_labeled.round().astype("Int64").astype(str)
param_labeled = param_labeled.round().astype("Int64").astype(str)
arg_labeled = arg_labeled.round(3)

arg_labeled = append_improvement_column(arg_labeled, arg_improvements)
depth_labeled = append_improvement_column(depth_labeled, depth_improvements)
param_labeled = append_improvement_column(param_labeled, param_improvements)

# 合并最终表格
merged_with_improvement = pd.concat([arg_labeled, depth_labeled, param_labeled])
merged_with_improvement.to_pickle("table_2.pkl")

# 显示结果
merged_with_improvement

Benchmark             F1      F2      F3      F4       K1       K2       K3  \
        method                                                                
ARG     HEA       16.842  63.123  71.239  96.073  767.072  464.644   830.25   
        P-QAOA    16.343  39.138  49.857  65.448  620.239  367.574  572.005   
        Choco-Q    0.087   0.197     0.2   0.201    0.681    0.891    0.603   
        Rasengan   0.001   0.076   0.023   0.108    0.006    0.793    0.121   
Depth   HEA           46      91     121     156       56       91      106   
        P-QAOA        87     148     174     207      180      253      287   
        Choco-Q      507    1888    2688    3848     1325     3527     4391   
        Rasengan      34      49      60      57       85       87       87   
#Params HEA           90     225     315     420      120      225      270   
        P-QAOA        10      10      10      10       10       10       10   
        Choco-Q       10      10      10      10       10       10       10   
        Rasengan       3      19      34      67        4        8       19   

Benchmark              K4      J1       J2       J3       J4       S1  \
        method                                                          
ARG     HEA       716.527  51.108  258.144  181.581  455.143  747.004   
        P-QAOA    604.227  52.243  180.667  153.871  294.547  530.624   
        Choco-Q     1.832   0.033    0.624    0.431    1.286    10.21   
        Rasengan    1.306   0.002    0.549    0.241    1.094    0.617   
Depth   HEA           121      51       66       86      106       61   
        P-QAOA        326     110      138      168      192      207   
        Choco-Q      5267    1012     2047     3117     4622      826   
        Rasengan       87      61       73       66       72       23   
#Params HEA           315     105      150      210      270      135   
        P-QAOA         10      10       10       10       10       10   
        Choco-Q        10      10       10       10       10       10   
        Rasengan       23       3        6       18       18        8   

Benchmark               S2        S3        S4       G1       G2       G3  \
        method                                                              
ARG     HEA       1402.251  1649.083  1623.916  200.165  447.039  550.923   
        P-QAOA     709.235  1388.615  1088.422  136.228    327.1  413.985   
        Choco-Q       6.62    11.907    37.685    0.239    0.769     1.36   
        Rasengan     0.794    17.045    16.392    0.223    0.638    0.406   
Depth   HEA             76        96       116       76       91      116   
        P-QAOA         283       411       476      260      309      744   
        Choco-Q       1570      1796      2642     1878     1650     3206   
        Rasengan        30        21        22       55       64       38   
#Params HEA            180       240       300      180      225      300   
        P-QAOA          10        10        10       10       10       10   
        Choco-Q         10        10        10       10       10       10   
        Rasengan        18        25        44        8        4       20   

Benchmark              G4 improvement  
        method                         
ARG     HEA       532.288    9185.523  
        P-QAOA    265.337      7707.8  
        Choco-Q     1.175      12.574  
        Rasengan    0.891        <NA>  
Depth   HEA           136        1.99  
        P-QAOA        786       6.714  
        Choco-Q      3520      49.068  
        Rasengan       62        <NA>  
#Params HEA           360      19.273  
        P-QAOA         10       1.113  
        Choco-Q        10       1.113  
        Rasengan       16        <NA>